In [2]:
!pip install emoji

     |████████████████████████████████| 51kB 2.1MB/s eta 0:00:011
  Created wheel for emoji: filename=emoji-0.5.4-cp37-none-any.whl size=42177 sha256=cf3a2e56cf1d27ddccacda1022a397884e7c01167cbec03f246e5bb65755d618
  Stored in directory: /home/jovyan/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import re
import emoji

In [4]:
with open('telegram.txt', 'r') as telegram:
    telegram_data = telegram.readlines()
# checking the messages
telegram_data[-10:]

["29.10.2019 15:53:48, Steven Tey: I'll talk to you later!! 😘\n",
 '29.10.2019 15:53:54, Steven Tey: Good later my love ❤️😞\n',
 '29.10.2019 15:53:56, Anzhelika Korolova: Good later❤️😘\n',
 '29.10.2019 15:53:57, Steven Tey: [[?messageMediaUnsupported]]\n',
 '29.10.2019 15:53:59, Anzhelika Korolova: [[?messageMediaUnsupported]]\n',
 '29.10.2019 15:53:59, Steven Tey: [[?messageMediaUnsupported]]\n',
 "29.10.2019 15:54:02, Anzhelika Korolova: [[😴 Sticker, size 17'458 bytes]]\n",
 "29.10.2019 15:54:02, Steven Tey: [[😴 Sticker, size 17'718 bytes]]\n",
 "29.10.2019 16:20:58, Steven Tey: [[Video, size 4'804'352 bytes]]\n",
 '29.10.2019 16:21:06, Steven Tey: [[Photo]]This is so creepy wth 😭🙈😵😵😵']

In [5]:
def remove_emoji(text, remove_components=False):
    '''
    Script to remove emojis from string, retrieved from:
    https://stackoverflow.com/a/51785357
    '''
    cleaned = emoji.get_emoji_regexp().sub(u'', text)
    if remove_components:
        cleaned = emoji_components.sub(u'', cleaned)
    return cleaned

def strip_text(text):
    '''
    Process the text further
    '''
    noEmojiMessage = remove_emoji(text)
    # Remove any links in the message
    noLinksMessage = re.sub(r'^https?:\/\/.*[\r\n]*', '', noEmojiMessage, flags=re.MULTILINE)
    # Remove any email addresses in the message
    processed_line = noLinksMessage.replace('\S*@\S*\s?',' ')
    first_colon = processed_line.find(': ') + 1
    just_message = processed_line[first_colon:].strip()
    sender_name = processed_line[:first_colon]
    return processed_line, first_colon, just_message, sender_name

def progressBar(current, total, barLength = 20):
    '''
    Standard progress bar algorithm retrieved from https://stackoverflow.com/a/37630397
    '''
    percent = float(current) * 100 / total
    arrow   = '-' * int(percent/100 * barLength - 1) + '>'
    spaces  = ' ' * (barLength - len(arrow))

    print('Processing the data: [%s%s] %d %%' % (arrow, spaces, percent), end='\r')


def get_Messages(message_data):
    '''
    Main function to process the data:
    - message_data: the txt file with all the extracted messages
    '''
    responseDictionary = dict()
    myMessage, otherPersonsMessage, currentSpeaker = "","",""
    forbidden_list = ["[[Photo", "[[Video", "[[Voice", "[[GIF", "[[Webpage]]", 
                          "[[?messageMediaUnsupported]]", "{{FWD", "{{GIF", "[[Geo"
                          "[[Document", ">>incoming call", ">>outgoing call", 
                          ">>missed call", ">>canceled call"]
    startMessageIndex = 0
    process_count = 0
    for idx, line in enumerate(message_data):
        processed_line, first_colon, just_message, sender_name = strip_text(line)
        if len(just_message) < 3:
            continue
        elif bool([ele for ele in forbidden_list if(ele in just_message)]) == True:
            continue
        elif "[[" and "Sticker" in just_message:
            continue
        else:
            # if the current message is from me
            if "Steven Tey" in sender_name: 
                if not myMessage:
                    # Get the first message I sent 
                    # (if I send multiple messages in a row)
                    startMessageIndex = idx - 1
                # add current message to myMessage cache
                myMessage = myMessage + ' ' + just_message
            elif myMessage: # if I'm not the one speaking 
                for counter in range(startMessageIndex, 0, -1):
                    currentLine = message_data[counter]
                    processed_line_other, first_colon_other, just_message_other, sender_name_other = strip_text(currentLine)
                    if len(just_message_other) < 3:
                        continue
                    elif bool([ele for ele in forbidden_list if(ele in just_message)]) ==  True:
                        continue
                    elif "[[" and "Sticker" in just_message_other:
                        continue
                    else:
                        if not currentSpeaker:
                            # If the currentSpeaker variable is empty, 
                            # assign Anzhelika as current speaker
                            if "Anzhelika" in sender_name_other:
                                currentSpeaker = "Anzhelika"
                        # Once there is a current speaker and the 
                        # otherPersonsMessage variable is not empty 
                        elif ("Anzhelika" not in sender_name_other and otherPersonsMessage):
                            # A different person started speaking, so now I know that the
                            # first person's message is done
                            responseDictionary[otherPersonsMessage] = myMessage
                            break
                        # But if those two conditions don't hold
                        # add current message to otherPersonsMessage cache
                        otherPersonsMessage = just_message_other + ' ' + otherPersonsMessage
                # clear caches at the end of the loop
                myMessage, otherPersonsMessage, currentSpeaker = "","",""  
        process_count += 1
        progressBar(process_count, len(message_data), barLength = 30)
    return responseDictionary       

In [6]:
messagesDict = get_Messages(telegram_data)

In [ ]:
for key, value in messagesDict.items():
    if (not key.strip() or not value.strip()):
        # If there are empty strings
        continue
    print(f'Prompt: {key.strip()}\n Response: {value.strip()}\n')

Prompt: Is it that cute girl
 Response: he's super smart though right?? I didn't know that you took CS110

Prompt: Yeah he is smart No seriously
 Response: ahaha

Prompt: “Cute”
 Response: no it's Anh it's a guy

Prompt: Who is that
 Response: a guy from my class la he smart

Prompt: you are smart too
 Response: want me to give you his whole backstory he grew up in Saigon his grandfather fought in WWII

Prompt: Where his parents work
 Response: he was a army general in the guerrila war against the US

Prompt: What’s his major What he is planning to do in 5 years Hahhaha
 Response: HAHAHAHAHAHAHAHHAHAHAHAHAH You crazy

Prompt: I love you so much more !!!️
 Response: so early?

Prompt: Thank you so much for being with me Baby I’m sleeping
 Response: HAHAHAHA

Prompt: Isabele and Ira are too
 Response: you are sleeping okay okay sorryyy la

Prompt: It’s almost 11:30pm
 Response: just wanted to hear your voice OHHH SHIT

Prompt: We can call tomorrow!!
 Response: wow that shit is tricky wew

In [10]:
import numpy as np

print('Saving messages data into a dictionary...')
np.save('responseDict.npy', messagesDict)
print('Done!')

print('Converting dictionary into a txt file...')
messagesFile = open('responseData.txt', 'w')
for key, value in messagesDict.items():
    if (not key.strip() or not value.strip()):
        # If there are empty strings
        continue
    messagesFile.write(key.strip() + value.strip())
print('Done! Filed saved at responseData.txt')

Saving messages data into a dictionary...
Done!
Converting dictionary into a txt file...
Done! Filed saved at responseData.txt
